In [1]:
from Bio import Entrez
import pickle
from lib.Publications import Publication

# Always tell NCBI who you are
Entrez.email = input("Email: ")

Email:  mats.topel@marine.gu.se


# Custom functions

In [2]:
# Store the search results locally
record_cash = {}

# Download individual records
def get_record(pubmedID):
    # Check if the publication has already been downloaded
    try:
        return record_cash[pubmedID]
    except KeyError:
        handle = Entrez.efetch(db="pubmed", \
                               rettype="text", \
                               id=pubmedID)
        result = Entrez.read(handle)
        
        # Update the cashed results
        record_cash[pubmedID] = result
        return result

# Print progress
def progress(x = None, y = None):
    # Print progress
    prog = str(x) + "/" + str(y)
    print(prog, end="\r")

# Query string

In [3]:
# Some usefull search tags to use:
# https://pubmed.ncbi.nlm.nih.gov/help/#search-tags
#
# All Fields [all]
# Author [au]
# Journal [ta]
# PMID [pmid]
# Publication Type [pt]
# Text Words [tw]
# Title [ti]
# Title/Abstract [tiab]


# [Author]
# [Journal]
# [Keyword]
# [Title]
# [pt] (Pulication type)
# Publication types: https://pubmed.ncbi.nlm.nih.gov/help/#publication-types
#query = "reproducible AND eDNA"
#query = "eDNA AND metabarcoding"
query = '("eDNA" OR "environmental DNA" OR "metabarcoding" OR "eRNA" OR "environmental RNA") AND ("biodiversity" OR "species richness" OR "monitoring" OR "biomonitoring") AND ("high throughput sequencing" OR "HTS" OR "throughput")'

# Search PubMed

In [4]:
# DevNote: Check the default retmax value
handle = Entrez.esearch(db="pubmed", \
                        term=query, \
                        retmax=100000)

record = Entrez.read(handle)

# Explore the result

In [5]:
'''
for pubmedID in record["IdList"]:
    x = get_record(pubmedID)
    break
    
    
# Title
print(x["PubmedArticle"][0]["MedlineCitation"]["Article"]["ArticleTitle"])

# Keywords
keywords = []
for i in x["PubmedArticle"][0]["MedlineCitation"]["KeywordList"][0]:
    keywords.append(str(i))
print(keywords)

# Abstract
x["PubmedArticle"][0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][0]

# DOI
x["PubmedArticle"][0]["MedlineCitation"]["Article"]["ELocationID"][0].title()

# eid

# pii

# Journal
print(x["PubmedArticle"][0]["MedlineCitation"]["Article"]["Journal"]["Title"])

# Date
y = x["PubmedArticle"][0]["MedlineCitation"]["Article"]["ArticleDate"][0]
print(y["Year"] + "-" + y["Month"] + "-" + y["Day"])
'''

'\nfor pubmedID in record["IdList"]:\n    x = get_record(pubmedID)\n    break\n    \n    \n# Title\nprint(x["PubmedArticle"][0]["MedlineCitation"]["Article"]["ArticleTitle"])\n\n# Keywords\nkeywords = []\nfor i in x["PubmedArticle"][0]["MedlineCitation"]["KeywordList"][0]:\n    keywords.append(str(i))\nprint(keywords)\n\n# Abstract\nx["PubmedArticle"][0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][0]\n\n# DOI\nx["PubmedArticle"][0]["MedlineCitation"]["Article"]["ELocationID"][0].title()\n\n# eid\n\n# pii\n\n# Journal\nprint(x["PubmedArticle"][0]["MedlineCitation"]["Article"]["Journal"]["Title"])\n\n# Date\ny = x["PubmedArticle"][0]["MedlineCitation"]["Article"]["ArticleDate"][0]\nprint(y["Year"] + "-" + y["Month"] + "-" + y["Day"])\n'

In [6]:
# Display the number of results found for this search string
print(f"Number of records: {len(record['IdList'])}")

Number of records: 417


# Explore the result in Pandas

In [7]:
# import pandas as pd

# Explore keywords

In [8]:
'''
keywords = []
prog = 1

for pubmedID in record["IdList"]:
    x = get_record(pubmedID)
    try:
        for i in x["PubmedArticle"][0]["MedlineCitation"]["KeywordList"][0]:
            keywords.append(str(i))
    except IndexError:
        pass
    # Print progress
    progress(prog, len(record["IdList"]))
    prog += 1
    
# Count the keywords
count = {}
for i in keywords:
    i = str(i)
    try:
        count[i] += 1
    except:
        count[i] = 1

# Turn the dictionary of keywords into a Pandas dataframe
kw = pd.DataFrame.from_dict(count, orient='index')

# Display the 20 top keywords
kw.sort_values(0, ascending=False).head(20)
'''

'\nkeywords = []\nprog = 1\n\nfor pubmedID in record["IdList"]:\n    x = get_record(pubmedID)\n    try:\n        for i in x["PubmedArticle"][0]["MedlineCitation"]["KeywordList"][0]:\n            keywords.append(str(i))\n    except IndexError:\n        pass\n    # Print progress\n    progress(prog, len(record["IdList"]))\n    prog += 1\n    \n# Count the keywords\ncount = {}\nfor i in keywords:\n    i = str(i)\n    try:\n        count[i] += 1\n    except:\n        count[i] = 1\n\n# Turn the dictionary of keywords into a Pandas dataframe\nkw = pd.DataFrame.from_dict(count, orient=\'index\')\n\n# Display the 20 top keywords\nkw.sort_values(0, ascending=False).head(20)\n'

# Explore abstracts in html format

In [9]:
publications = []

prog = 1
with open("original_abstracts_PubMed.html", "w") as file:
    for pubmedID in record["IdList"]:
        
        progress(prog, len(record["IdList"]))
        
        paper = get_record(pubmedID)
        
        # Title
        title = "<h1>" + paper["PubmedArticle"][0]["MedlineCitation"]["Article"]["ArticleTitle"] + "</h1>" + "\n"
        title_obj= paper["PubmedArticle"][0]["MedlineCitation"]["Article"]["ArticleTitle"]
        
        # DOI
        try:  
            doi = "<p><a href=\"https://doi.org/" + \
                  paper["PubmedArticle"][0]["MedlineCitation"]["Article"]["ELocationID"][0].title() + \
                  "\">" + \
                  "doi:" + \
                  paper["PubmedArticle"][0]["MedlineCitation"]["Article"]["ELocationID"][0].title() + \
                  "</a></p>" + \
                  "\n"
            doi_obj = paper["PubmedArticle"][0]["MedlineCitation"]["Article"]["ELocationID"][0].title()
        except:
            doi = "<p>No DOI</p>"
            doi_obj = None
        
        # Keywords
        try:
            keywords = []
            for i in paper["PubmedArticle"][0]["MedlineCitation"]["KeywordList"][0]:
                keywords.append(str(i))
            #keywords = paper["PubmedArticle"][0]["MedlineCitation"]["KeywordList"][0]
        except:
            keywords = "<p>No keywords</p>"
        
        # Abstract
        try:
            abstract = "<p>" + \
                       paper["PubmedArticle"][0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][0] + \
                       "</p>" + \
                       "\n"
            abstract_obj = paper["PubmedArticle"][0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][0]
        except KeyError:
            abstract = "<p>No Abstract</p>"
            abstract_obj = None
            
        string = title + doi + str(keywords) + abstract

        publications.append(Publication(title = str(title_obj), doi = str(doi_obj), abstract = str(abstract_obj)))

        prog += 1
        file.write(string)

# Save result to binary file

In [10]:
# Save the result to a binary file, and analyse it together with data from other searches.
pickle.dump(publications, open("pubmed.p" ,"wb"))